# Šachy - cvičení 02

předcházející část: [chess_exercise01.ipynb](chess_exercise01.ipynb)

Díky předcházející části by měl být funkční tento kód: 


In [2]:
import chess_exercise01 as chess
ch = chess.Chessboard("RnBkQbNr")
f = open("pokus.htm", "w")
html = ch.get_html()
f.write(html)
f.close()

Jeho výsledkem by měl být HTML soubor [pokus.htm](pokus.htm) zobrazující zadanou pozici figur. 

## Serverová aplikace v Pythonu

Naším nejbližším dalším cílem bude vytvoření jednoduché serverové aplikace, která bude vracet webovou stránku s obrázkem šachovnice a na ní zobrazených figur. Rozmístění figur bude odpovídat řetězci, který zadáme v URL. Např.: <adresa serveru>/chessboard/3KQ by měla vrátit šachovnici s bílým králem a dámou ve své výchozí pozici. 

### pythonanywhere.com

Server vytvoříme jednoduše pomocí služby https://www.pythonanywhere.com. Zde si zřídíme účet - stačí ten zadarmo (demonstruji na účtu jardacarda). Po prvním přihlášení není vytvořena a spuštěna žádná webová aplikace. Na výchozí stránce (dashboard) vybereme volbu web a následně Add new web app. U neplaceného účtu můžeme vytvořit pouze aplikaci běžící v doméně pythonanywhere.com, takže volíme Next. Nyní si můžeme vybrat, jakého typu naše aplikace bude. 

### Bottle

Třeba se časem dopracujeme i k velmi rozšířenému webovém frameworku Django, ale pro jednoduchost a demonstraci základních principů použijeme Bottle. V další obrazovce vybereme verzi Pythonu, ve které budeme aplikaci vytvářet. Můžeme zvolit nejvyšší trojkovou verzi, která je zde k dispozici. Dále jsme dotazování na jméno a umístění naší aplikace. Opět volíme nabízenou variantu tedy /home/&lt;jméno&gt;/mysite/bottle_app.py. 

Za chvilku budete informováni, že byla aplikace spuštěna a že je dostupná na adrese &lt;jméno&gt;.pythonanywhere.com. Když tuto adresu otevřeme v prohlížeči, zobrazí se Hello from Bottle! 

### WSGI

Jak k tomu zázraku došlo? Pythonanywhere poskytuje služby svého webového serveru prostřednictvím rozhraní WSGI, což je vlastně standradizovaný interface mezi webovým serverem a naší pythoní aplikací. Konfiguraci tohoto rozhraní pythonanywhere udělal za nás na základě volby frameworku Bottle a cesty k aplikaci. Konfigurační soubor najdeme v adresáři /var/www, jeho název bude <jméno>_pythonanywhere_com_wsgi.py. Pokud budeme chtít změnit jméno nebo umístění naší aplikace musíme v tomto souboru změnit project_home eventuálně from ... import application: 

In [11]:
# This file contains the WSGI configuration required to serve up your
# web application at http://<your-username>.pythonanywhere.com/
# It works by setting the variable 'application' to a WSGI handler of some
# description.
#
# The below has been auto-generated for your Bottle project
import bottle
import os
import sys

# add your project directory to the sys.path
project_home = u'/home/jardacarda/mysite'
if project_home not in sys.path:
    sys.path = [project_home] + sys.path

# make sure the default templates directory is known to Bottle
templates_dir = os.path.join(project_home, 'views/')
if templates_dir not in bottle.TEMPLATE_PATH:
    bottle.TEMPLATE_PATH.insert(0, templates_dir)

# import bottle application
from bottle_app import application


### bottle_app.py

Výchozí ukázková aplikace bottle_app.py v adresáři ~/mysite bude vypadat asi takhle: 

In [12]:
# A very simple Bottle Hello World app for you to get started with...
from bottle import default_app, route

@route('/')
def hello_world():
    return 'Hello from Bottle!'

application = default_app()

I když plně neporozumíme významu tohoto kódu, tak je asi jasné, že funkce hello_world() vrací řetězec, který je pak zobrazen browserem. Když se podíváme na zdrojový kód stránky v browseru (ctrl + u), tak vidíme, že tam skutečně je přesně to, co ta funkce vrací. Nic víc a nic míň. Zkuste funkci upravit, aby vrátila validní HTML kód. Třeba takhle: 

In [13]:
def hello_world(): 
    html = """<!DOCTYPE html>
<html>
	<head>
		<title>my bottle page</title>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	</head>
	<body>
        <h1>Hello from Bottle!</h1>
        <p>The bottle is in the Python language not in the sea...</p>
	</body>
</html>
"""
    return html

Nezapomeňte upravený kód uložit a spustit reload aplikace. Buď v sekci Web, kde jste aplikaci vytvářeli nebo pomocí tlačítka vpravo nahoře přímo v editoru. Pokud tedy editor ze sekce Files používáte. 

### dekorátor

Naše funkce se může jmenovat libovolně. Rozhodující je, že před její definicí je kouzelná konstrukce @route('/'), které se říká [dekorátor](chess_exercise_decorator.ipynb). Funkce route, kterou dekorujeme naši funkci má parametr - v tomto případě "/". Co to znamená? Že ta naše funkce se spustí v případě, že z prohlížeče přijde požadavek na root ("/") našeho webu. Pokud bychom chtěli zareagovat na jinou adresu, změníme parametr funkce route. Před naši funkci můžeme dát dekorátorů @route víc, takže může reagovat na různé URL. Pokud chceme na jiné URL reagovat jinak, definujeme další funkci, atu opět odekorujeme pomocí @route. Může to vypadat třeba takhle: 

In [14]:
# my Bottle application
from bottle import default_app, route

@route('/')
@route('/home')
@route('/home/')
def home():
    return "home site"

@route('/chess')
@route('/chess/')
def chess(): 
    return "chess exercises"

application = default_app()
